In [5]:
#importing all the necessary libraries
import urllib.request
from bs4 import BeautifulSoup
import ssl
import lxml
import requests
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

In [ ]:
#ignoring the ssl certificate errors
import ssl
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [ ]:
#scraping multiple websites from the list of sites given and storing the scraped data in different text files

def scrape(u):      #function to scrape the data using beautifulsoup
    url = i
    print(url)
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text,'html.parser')
    return soup

def make_file(data,fname):      #function to store the scraped data in different scraped files    
    filename = str(fname)+".txt"
    f = open(filename,'a+')
    title = data.find('title').text.strip()     #scraping the title
    print(title)
    f.write(title+'\n')
        
    art_text = data.find_all('p')       #scraping the body
    for j in art_text:
       #print(i.text.strip())
        f.write(j.text.strip())
    f.close()

df = pd.read_excel('Input.xlsx')

for i in df['URL']:
    scraped_data = scrape(i)
    df2 = df.loc[df['URL']== i,'URL_ID']
    p = list(df2.values)
    id = p.pop(0)
    make_file(scraped_data,id)




In [46]:
#reading the postive and negative words text files and storing them in two different lists
f_pos = open('positive-words.txt','r')
content = f_pos.readlines()
positive_words = []
for i in content:
    positive_words.append(i.rstrip())      
#print(positive_words)
f_pos.close()

f_neg = open('negative-words.txt','r',encoding="ISO-8859-1")
content1 = f_neg.readlines()
negative_words = []
for i in content1:
    negative_words.append(i.rstrip())
f_neg.close()

In [48]:
#removing stopwords using nltk package as mentioned in the obejctives.docx and calculating the variables.
def remove(i):
    filename = str(i)+".txt"
    f = open(filename,'r')
    content = f.readlines()
    url_file_content = ""
    for j in content:
        url_file_content = url_file_content+" "+j
        
    stop_words = set(stopwords.words('english'))    

    word_tokens = word_tokenize(url_file_content)

    modified_content = []
    for w in word_tokens:
        if w not in stop_words:         
            modified_content.append(w)
    return modified_content,url_file_content

#calculating the variables for all the files and storing them in separate lists.

word_count = []
positive_score = []
negative_score = []
polarity_Score = []
subjectivity_score = []
complex_word_count = []
avg_sentence_length = []
perc_of_complex_words = []
fog_index = []
avg_words_per_sent = []
personal_pronoun_count = []
avg_word_length = []

for i in df['URL_ID']:
    modified_content,url_file_content = remove(i) 
    wc = len(modified_content)
    word_count.append(wc)

    p_score = 0
    n_score = 0

#print(url_file_content)
    sent = sent_tokenize(url_file_content)
    sentence_count = len(sent)   #calculating the sentence length
    complex_word = []
    personal_pronoun = []
    char = 0
    char_count = 0

    for word in url_file_content.split():
        if word in positive_words:
            p_score = p_score + 1           #calculating positive score for each file
        if word in negative_words:
            n_score = n_score - 1           #calculating negative score for each file

        #calculating number of syllable for each word in a file
        syllable_count=0

        for w in word:
            #print(j)
            if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U'):
                syllable_count=syllable_count+1         
                #print(syllable_count)
        if syllable_count>2:        
            complex_word.append(word)       #creating a list of complex words for each file
        
        #calculating the count of personal pronouns in each file
        if (word=='I' or word=='i' or word=='we' or word=='We'or word=='my' or word=='My' or word=='ours' or word=='Ours' or word=='us' or word=='Us'):
            personal_pronoun.append(word)       
        for c in word:
            char = char+1
        char_count = char_count+char

    #appending all the calculated variables for each file and appending them in separate lists.
    #These lists, at the end will contain the value of variables of all the files. 
    avg_wlen = char_count/wc
    avg_word_length.append(avg_wlen)

    personal_pcount = len(personal_pronoun)
    personal_pronoun_count.append(personal_pcount)

    com_len = len(complex_word)
    complex_word_count.append(com_len)

    avg_sent_len = wc/sentence_count
    avg_sentence_length.append(avg_sent_len)

    perc_of_complex = com_len/wc
    perc_of_complex_words.append(perc_of_complex)

    f_index = 0.4 * (avg_sent_len + perc_of_complex)
    fog_index.append(f_index)

    avg_words = wc/sentence_count
    avg_words_per_sent.append(avg_words)

    positive_score.append(p_score)
    negative_score.append(n_score)

    polar_Score = ((p_score - n_score)/((p_score + n_score) + 0.000001))
    polarity_Score.append(polar_Score)

    sub_score = (p_score + n_score)/ ((wc) + 0.000001)
    subjectivity_score.append(sub_score)





In [49]:
#writing the calculated variables in output data structure.xlxs as mentioned in the instructions.
old_df = pd.read_excel('Output Data Structure.xlsx')  
new_df = pd.DataFrame({'POSITIVE SCORE':positive_score,'NEGATIVE SCORE':negative_score,'POLARITY SCORE':polarity_Score,'SUBJECTIVITY SCORE':subjectivity_score,'AVG SENTENCE LENGTH':avg_sentence_length,'PERCENTAGE OF COMPLEX WORDS':perc_of_complex_words,'FOG INDEX':fog_index,
            'AVERAGE NUMBER OF WORDS':avg_words_per_sent,'COMPLEX WORD COUNT' : complex_word_count, 'WORD_COUNT' : word_count,'PERSONAL PRONOUN COUNT':personal_pronoun_count,'AVERAGE WORD LENGTH':avg_word_length})
frames = [old_df,new_df]
result = pd.concat(frames, axis=1)
result.to_excel('Output Data Structure.xlsx',sheet_name="sheet_1", index=False)